In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator, load_img
import optuna
import math
import os
import time

Using TensorFlow backend.
/home/mason/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mason/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mason/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mason/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [2]:
path = '/home/mason/Documents/school/machine_learning/projects/applied_project/0xDEADBEEFCAFE/python/pneumonia/creating_models_optuna'
data_path = path + "/../data"
test_data_filepath = data_path + "/test"
train_data_filepath = data_path + "/train"
val_data_filepath = data_path + "/val"

BATCH_SIZE = 10
NUM_EPOCHS = 20
NUM_TRAIN_SAMPLES = 1341 + 3875 # Normal + Pneumonia
NUM_VAL_SAMPLES = 8 + 8 # Normal + Pneumonia

INPUT_SHAPE = (255, 255, 3)

In [3]:
def objective(trial):
    train_gen = ImageDataGenerator( rescale=1./255,
                                    shear_range= 0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

    val_gen = ImageDataGenerator(rescale=1. / 255)

    train_set = train_gen.flow_from_directory(  train_data_filepath,
                                                target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
                                                batch_size=BATCH_SIZE,
                                                class_mode='binary')

    val_set = val_gen.flow_from_directory(  val_data_filepath,
                                            target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
                                            batch_size=BATCH_SIZE,
                                            class_mode='binary')

    test_gen = ImageDataGenerator(rescale= 1./255)
    test_set = test_gen.flow_from_directory(  test_data_filepath,
                                                target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
                                                batch_size=BATCH_SIZE,
                                                class_mode='binary')

    model = Sequential()
    
    # First Convolutional Layer
    model.add(
        Conv2D( filters=trial.suggest_categorical('filters_1', [16, 32, 64, 128]),
                kernel_size=trial.suggest_categorical('kernel_size_1', [2, 3, 4]),
                input_shape=INPUT_SHAPE,
                activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(trial.suggest_uniform('dropout_1', 0.0, 1.0)))

    # Second Convolutional Layer
    model.add(
        Conv2D( filters=trial.suggest_categorical('filters_2', [16, 32, 64, 128]),
                kernel_size=trial.suggest_categorical('kernel_size_2', [2, 3, 4]),
                activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(trial.suggest_uniform('dropout_2', 0.0, 1.0)))

    # Third Convolutional Layer
    model.add(
        Conv2D( filters=trial.suggest_categorical('filters_3', [16, 32, 64, 128]),
                kernel_size=trial.suggest_categorical('kernel_size_3', [2, 3, 4]),
                activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(trial.suggest_uniform('dropout_3', 0.0, 1.0)))

    # Fourth Convolutional Layer
    if trial.suggest_categorical('Fourth Layer', [True, False]):
        model.add(
            Conv2D( filters=trial.suggest_categorical('filters_4', [16, 32, 64, 128]),
                    kernel_size=trial.suggest_categorical('kernel_size_4', [2, 3, 4]),
                    activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(trial.suggest_uniform('dropout_4', 0.0, 1.0)))

    # Feed forward network ----------
    model.add(Flatten())
    model.add(Dense(trial.suggest_categorical('dense_layer_nodes_1', [16, 32, 64]), activation='relu'))

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.fit_generator(train_set,
                        steps_per_epoch=int(NUM_TRAIN_SAMPLES/BATCH_SIZE),
                        nb_epoch=NUM_EPOCHS,
                        validation_data=val_set,
                        validation_steps=math.ceil(NUM_VAL_SAMPLES / BATCH_SIZE))
  
    accuracy = model.evaluate_generator(test_set)[1]
    
    model.save(path + '/' + 'models/model_' + str(int(accuracy * 100 * 100)) + '_' + str(int(math.ceil(time.time()))) + '.h5')

    return -accuracy

In [4]:
study = optuna.create_study()
study.optimize(objective, n_trials = 20)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
/home/mason/.local/lib/python3.6/site-packages/ipykernel_launcher.py:74: UserWarning:

The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.

/home/mason/.local/lib/python3.6/site-packages/ipykernel_launcher.py:74: UserWarning:

Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=104, validation_data=<keras.pre..., validation_steps=1, epochs=20)`


Epoch 1/20
[W 2019-12-05 19:55:43,776] Set

ResourceExhaustedError: OOM when allocating tensor with shape[50,253,253,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node training/Adam/gradients/max_pooling2d_1/MaxPool_grad/MaxPoolGrad}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [0]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_contour(study)
optuna.visualization.plot_intermediate_values(study)
optuna.visualization.plot_slice(study)
optuna.visualization.plot_parallel_coordinate(study)